# `APP` - Full Code:
For specific function explanation see the **Testing** section below:

# Testing:
We will test each function singularly to demonstrate how the output looks. This will also allow users to try and modify the function in case they want to add extra elements to their request output.

## Importing required libreries:

In [1]:
import pandas as pd
from flask import Flask,jsonify, json, request, render_template
import requests
import itertools
#import plotly.graph_objs as go
#import plotly.plotly as py
#from plotly.utils import PlotlyJSONEncoder
from pprint import pprint
#import requests_cache

## Defining support Function `clean_col` :

In [2]:
def clean_col(inputlst):
    col_list = []
    for col in inputlst:
        col = col.strip()
        col = col.lower()
        col = col.replace(" ", "_")
        col = col.replace("(", "")
        col = col.replace(")", "")
        col = col.replace("-", "_")
        col = col.strip()
        col_list.append(col)
    return col_list

## 1. `Get_content`:
`get_content` allows the user to extract all the records from the Police API. Nevertheless, these records are not layed out in a functional format and can be difficult to interpret if the users do not have sufficient experience with dictionary and json files.

In [3]:
def get_content(date, n_records): #"get_records" has two parameters:
                                      #1. "date" parameter
                                      #2. "n_records" paramter, which can be either All or an interger
    
    #Setting the API url:
    crime_url_template = 'https://data.police.uk/api/outcomes-at-location?lat={lat}&lng={lng}&date={data}'

    my_latitude = '51.509865' #The latitude of London City
    my_longitude = '-0.118092' #The longitute of London City
    extract_date = str(date) #Converting the "date" parameter given by the path to a string
    year = extract_date[0:4] #Slicing date in "year" (ex. "2018")
    month = extract_date[4:len(extract_date)] #Slicing date in "month" (ex. "11")
    my_date = year + "-" + month #Connecting "year" and "month" whilst adding a dash (ex. "2018-11")
   
    #Calling format on the API string to change {lat}, {lng}, {data} into the above-set paramters:
    crime_url = crime_url_template.format(
        lat = my_latitude,
        lng = my_longitude,
        data = my_date) 

    #Calling ".get" function to establish a connection with the Police API:
    resp = requests.get(crime_url)
    
    #If a "status_code" equal to 200 is returned then the resp in "json" format is returned:
    if resp.status_code == 200:
        all_crime_data = resp.json()

    #Else return the "status_code" of the respons:
    else:
        print(resp.status_code)

This is an example of the output of the `get_content` function:

In [4]:
get_content("201811")

TypeError: get_content() missing 1 required positional argument: 'n_records'

The `get_content` function is only a building block for the more advanced function that the app will have. Indeed, in the App there is no path that allows the users to call `get_content`. Nevertheless, you will notice that each of the function used in the app uses the same logic of `get_content` to extract the records from the API. Hence, it is useful to see what it does, especially if one wants to change the logic of one of the apps functions.

## 2. `Get Records`:
The `get_records` functions allows the users to extract all the records in a more polished format. Indeed this time each record has one single key and all the values are specified under that key. This removes abstraction and allows the users to benefit from a neat output when formulating his request. 

`get_records` allows for a more complex logic when dictating the `curl` request. It allows the user to decide either the amount of records he wants to extract or to extract all the records. Finally, it allows advices the user if the number of records he tried request are more than the the total amount of records. 

`get_records` takes in a `date` directly from the path specified in the `curl` request.

In [5]:
def get_records(date, n_records): #"get_records" has two parameters:
                                      #1. "date" parameter
                                      #2. "n_records" paramter, which can be either All or a number, any other value will return an error:
    
    #Setting the API url:
    crime_url_template = 'https://data.police.uk/api/outcomes-at-location?lat={lat}&lng={lng}&date={data}'

    my_latitude = '51.509865' #The latitude of London City
    my_longitude = '-0.118092' #The longitute of London City
    extract_date = str(date) #Converting the "date" parameter given by the path to a string
    year = extract_date[0:4] #Slicing date in "year" (ex. "2018")
    month = extract_date[4:len(extract_date)] #Slicing date in "month" (ex. "11")
    my_date = year + "-" + month #Connecting "year" and "month" whilst adding a dash (ex. "2018-11")
   
    #Calling format on the API string to change {lat}, {lng}, {data} into the above-set paramters:
    crime_url = crime_url_template.format(
        lat = my_latitude,
        lng = my_longitude,
        data = my_date) 

    #Calling ".get" function to establish a connection with the Police API:
    resp = requests.get(crime_url)
    
    #If a "status_code" equal to 200 is returned then the resp in "json" format is returned:
    if resp.status_code == 200:
        all_crime_data = resp.json()

    #Else return the "status_code" of the respons:
    else:
        print(resp.status_code)
    
    #Creating a set of list to assign each value for each entry in the dictionary (see Mini_Project.ipynb)
    codes = []
    procedures = []
    dates = []
    person_ids = []
    crime_categories = []
    location_types = []
    latitudes = []
    longitudes = []
    street_ids = []
    street_names = []
    contexts = []
    persistent_ids = []
    crime_ids = []
    location_subtypes = []
    months = []

    #Calling a loop for every entry in the dictionary that has been returned from the response:
    for i in range(len(all_crime_data)):

    #Assigning each "i" value for each iteration to a variable:
        code = all_crime_data[i]["category"]["code"]
        procedure = all_crime_data[i]["category"]["name"]
        date =  all_crime_data[i]["date"]
        person_id = all_crime_data[i]["person_id"]
        crime_category = all_crime_data[i]["crime"]["category"]
        location_type = all_crime_data[i]["crime"]["location_type"]
        latitude = all_crime_data[i]["crime"]["location"]["latitude"]
        longitude = all_crime_data[i]["crime"]["location"]["longitude"]
        street_id = all_crime_data[i]["crime"]["location"]["street"]["id"]
        street_name = all_crime_data[i]["crime"]["location"]["street"]["name"]
        context = all_crime_data[i]["crime"]["context"]
        persistent_id = all_crime_data[i]["crime"]["persistent_id"]
        crime_id = all_crime_data[i]["crime"]["id"]
        location_subtype = all_crime_data[i]["crime"]["location_subtype"]
        month = all_crime_data[i]["crime"]["month"]
        
    #Appending each variable to the respective list:
        codes.append(code)
        procedures.append(procedure)
        dates.append(date)
        person_ids.append(person_id)
        crime_categories.append(crime_category)
        location_types.append(location_type)
        latitudes.append(latitude)
        longitudes.append(longitude)
        street_ids.append(street_id)
        street_names.append(street_name)
        contexts.append(context)
        persistent_ids.append(persistent_id)
        crime_ids.append(crime_id)
        location_subtypes.append(location_subtype)
        months.append(month)
    
    #Creating a dataframe and using its vectorizaion properties to assign the remaning columns.
    df_final = pd.DataFrame(codes, columns = ["codes"])
    df_final["procedures"] = procedures
    df_final["dates"] = dates
    df_final["person_ids"] = person_ids
    df_final["crime_categories"] = crime_categories
    df_final["location_types"] = location_types
    df_final["latitudes"] = latitudes
    df_final["longitudes"] = longitudes
    df_final["street_ids"] = street_ids
    df_final["street_names"] = street_names
    df_final["contexts"] = contexts 
    df_final["persistent_ids"] = persistent_ids
    df_final["crime_ids"] = crime_ids
    df_final["location_subtypes"] = location_subtypes
    df_final["months"] = months
    #The Dataframe now holds every single entry of our response.
    
    #Converting the Dataframe to a dictionary:
    dictionary = df_final.to_dict(orient = "index")

    #If the second paramter of the function ("n_records") is equal to "None":
    if n_records == "All":
        
        #Return the all the records in dictionary in a json format.
        return dictionary
    
    
    #Else if the "n_records" is a interger stored as an string and is within the amounts of total records:
    elif int(n_records) in range(len(dictionary)):
        
        #Convert the string into an interger and use it to return the sliced dictionary
        n_record = int(n_records)
        return list(itertools.islice(dictionary.items(), 0, n_record))
    
    #In the case the "n_records" paramter is not "All" or a string stored as an interger that is bigger then then the total amount of records return a suggestion:
    else:
        return '<README>Do Not Panic! Your request has been successful. Unfortunatley the n_records paramter exeeds the amount of records in the dictionary. Try again by using either "All" in your path to retrive all records or inserting the amount of records you want to request.<README>'

### 2.1 Below we can visualise the three possible outcomes `get_records`:
All examples outputs are for a request which is send to the `all_crime_data/<date>/<n_records>` Path.

**1. Outcome** == User requests all records in the dictionary

In [6]:
get_records("201811", "All")

{0: {'codes': 'unable-to-prosecute',
  'procedures': 'Unable to prosecute suspect',
  'dates': '2018-11',
  'person_ids': nan,
  'crime_categories': 'violent-crime',
  'location_types': 'Force',
  'latitudes': '51.516165',
  'longitudes': '-0.102502',
  'street_ids': 587380,
  'street_names': 'On or near Green Arbour Court',
  'contexts': '',
  'persistent_ids': '18fb11f58eb36c4c914ffda35892d03e52c9f3670dc0c6967131ad83a6897df1',
  'crime_ids': 69668669,
  'location_subtypes': 'ROAD',
  'months': '2018-10'},
 1: {'codes': 'no-further-action',
  'procedures': 'Investigation complete; no suspect identified',
  'dates': '2018-11',
  'person_ids': nan,
  'crime_categories': 'other-theft',
  'location_types': 'Force',
  'latitudes': '51.519808',
  'longitudes': '-0.131669',
  'street_ids': 960740,
  'street_names': 'On or near Store Street',
  'contexts': '',
  'persistent_ids': '8828bf353c77bc05b360b7a4a2b0f411e660a9eb7d0d86db2663e678e739476f',
  'crime_ids': 69585340,
  'location_subtypes'

**2. Outcome** == User requests a specific amount of records which is **smaller** than the total amount of records in the dictionary:

In [92]:
get_records("201811", "50")

[(0,
  {'codes': 'unable-to-prosecute',
   'procedures': 'Unable to prosecute suspect',
   'dates': '2018-11',
   'person_ids': nan,
   'crime_categories': 'violent-crime',
   'location_types': 'Force',
   'latitudes': '51.516165',
   'longitudes': '-0.102502',
   'street_ids': 587380,
   'street_names': 'On or near Green Arbour Court',
   'contexts': '',
   'persistent_ids': '18fb11f58eb36c4c914ffda35892d03e52c9f3670dc0c6967131ad83a6897df1',
   'crime_ids': 69668669,
   'location_subtypes': 'ROAD',
   'months': '2018-10'}),
 (1,
  {'codes': 'no-further-action',
   'procedures': 'Investigation complete; no suspect identified',
   'dates': '2018-11',
   'person_ids': nan,
   'crime_categories': 'other-theft',
   'location_types': 'Force',
   'latitudes': '51.519808',
   'longitudes': '-0.131669',
   'street_ids': 960740,
   'street_names': 'On or near Store Street',
   'contexts': '',
   'persistent_ids': '8828bf353c77bc05b360b7a4a2b0f411e660a9eb7d0d86db2663e678e739476f',
   'crime_ids'

**3. Outcome** == User requests a specific amount of records which is **bigger** than the total amount of records in the dictionary:

In [94]:
get_records("201811", "2000")

'<README>Do Not Panic! Your request has been successful. Unfortunatley the n_records paramter exeeds the amount of records in the dictionary. Try again by using either "All" in your path to retrive all records or inserting the amount of records you want to request.<README>'

## 3. `Get_Code`:
The `get_code` function allows the users to extract the whole list of `category codes` **count** for one specific date. 

`get_codes` takes in a `date` directly from the path specified in the `curl` request.

In [88]:
def get_code(date):#"get_code" has only one paramter ("date") which is given in the path.
    
    #Setting the API url:
    crime_url_template = 'https://data.police.uk/api/outcomes-at-location?lat={lat}&lng={lng}&date={data}'

    my_latitude = '51.509865' #The latitude of London City
    my_longitude = '-0.118092' #The longitute of London City
    extract_date = str(date) #Converting the "date" parameter given by the path to a string
    year = extract_date[0:4] #Slicing date in "year" (ex. "2018")
    month = extract_date[4:len(extract_date)] #Slicing date in "month" (ex. "11")
    my_date = year + "-" + month #Connecting "year" and "month" whilst adding a dash (ex. "2018-11")
   
    #Calling format on the API string to change {lat}, {lng}, {data} into the above-set paramters:
    crime_url = crime_url_template.format(
        lat = my_latitude,
        lng = my_longitude,
        data = my_date) 

    #Calling ".get" function to establish a connection with the Police API:
    resp = requests.get(crime_url)
    
    #If a "status_code" equal to 200 is returned then the resp in "json" format is returned:
    if resp.status_code == 200:
        all_crime_data = resp.json()

    #Else return the "status_code" of the respons:
    else:
        return resp.status_code
    
    #Create an empty list ("location") and an empty dictionary ("consequences"):
    location = []
    consequences = {}
    
    #Running a loop which iterates n_times (n = to the total records in the response dictionary):
    for i in range(len(all_crime_data)):
        
    #For every single "i" record in the response dictionary:
    
        #1. Take the code value in the category section of the record:
        row = all_crime_data[i]["category"]["code"]
        
        #2. Append the value to the location list:
        location.append(row)
 
        #3. Count the apperance of each entry:
        if row in consequences: 
            consequences[row] = consequences[row] +1 
        else:
            consequences[row] = 1

    #Transform the dictionary with the count of the apperances of each entry into a DataFrame:
    df_consequences = pd.DataFrame(list(consequences.items()), columns=['consequence', 'count'])

    #Utilise the vectorised functionalities of DataFrames to compute a new percentage column:
    df_consequences["percentage"] = df_consequences["count"] / df_consequences["count"].sum()
    
    #Cleaning column font:
    df_consequences["consequence"] = clean_col(df_consequences["consequence"])
    
    #Sort values in aseconding order and convert the new DataFrame back to a dictionary:
    consequences_dict = df_consequences.sort_values("percentage", ascending = True).to_dict("index")
  
    #Return the dictionary in json format:
    return consequences_dict

This is an example of an output for a request which is send to the `/code_count/201811` Path.

In [89]:
get_code("201811")

{14: {'consequence': 'sent_to_crown_court',
  'count': 1,
  'percentage': 0.0006341154090044388},
 16: {'consequence': 'compensation',
  'count': 2,
  'percentage': 0.0012682308180088776},
 15: {'consequence': 'drugs_possession_warning',
  'count': 2,
  'percentage': 0.0012682308180088776},
 8: {'consequence': 'further_investigation_not_in_public_interest',
  'count': 4,
  'percentage': 0.0025364616360177552},
 13: {'consequence': 'penalty_notice_issued',
  'count': 8,
  'percentage': 0.0050729232720355105},
 9: {'consequence': 'unable_to_proceed',
  'count': 9,
  'percentage': 0.0057070386810399495},
 10: {'consequence': 'conditional_discharge',
  'count': 11,
  'percentage': 0.006975269499048827},
 12: {'consequence': 'community_penalty',
  'count': 16,
  'percentage': 0.010145846544071021},
 4: {'consequence': 'not_guilty',
  'count': 17,
  'percentage': 0.01077996195307546},
 7: {'consequence': 'suspended_sentence',
  'count': 17,
  'percentage': 0.01077996195307546},
 0: {'consequ

## 3. `Get_Loc`:
The `get_loc` function allows the users to extract the whole list of `crime_sublocation` **count** for one specific date. 

`get_loc` takes in a `date` directly from the path specified in the `curl` request.

In [96]:
def get_loc(date): #"get_loc" has only one paramter ("date") which is given in the path.
    
    #Setting the API url:
    crime_url_template = 'https://data.police.uk/api/outcomes-at-location?lat={lat}&lng={lng}&date={data}'

    my_latitude = '51.509865' #The latitude of London City
    my_longitude = '-0.118092' #The longitute of London City
    extract_date = str(date) #Converting the "date" parameter given by the path to a string
    year = extract_date[0:4] #Slicing date in "year" (ex. "2018")
    month = extract_date[4:len(extract_date)] #Slicing date in "month" (ex. "11")
    my_date = year + "-" + month #Connecting "year" and "month" whilst adding a dash (ex. "2018-11")
   
    #Calling format on the API string to change {lat}, {lng}, {data} into the above-set paramters:
    crime_url = crime_url_template.format(
        lat = my_latitude,
        lng = my_longitude,
        data = my_date) 

    #Calling ".get" function to establish a connection with the Police API:
    resp = requests.get(crime_url)
    
    #If a "status_code" equal to 200 is returned then the resp in "json" format is returned:
    if resp.status_code == 200:
        all_values = resp.json()

    #Else return the "status_code" of the respons:
    else:
        return resp.status_code

    #Create an empty list ("location") and an empty dictionary ("location_type"):
    location = []
    location_type = {}
    
    #Running a loop which iterates n_times (n = to the total records in the response dictionary):
    for i in range(len(all_values)):
        
    #For every single "i" record in the response dictionary:
    
        #1. Take the location_subtype value in the crime section of the record:
        row = all_values[i]["crime"]["location_subtype"]
         
        #2. Append the value to the location list:
        location.append(row)
        
        #3. Count the apperance of each entry:
        if row in location_type:
            location_type[row] =location_type[row] +1 
        else:
            location_type[row] = 1
    
    #Transform the dictionary with the count of the apperances of each entry into a DataFrame:
    df_location = pd.DataFrame(list(location_type.items()), columns=['location', 'count'])
    
    #Utilise the vectorised functionalities of DataFrames to compute a new percentage column:
    df_location["percentage"] = df_location["count"] / df_location["count"].sum()
    
    #Cleaning column strings:
    df_location["location"] = clean_col(df_location["location"])
    
    #Sort values in aseconding order and convert the new DataFrame back to a dictionary:
    loc_dict = df_location.sort_values("percentage", ascending = True).to_dict("index")
    
    #Return the dictionary in json format:
    return loc_dict

This is an example of an output for a request which is send to the `/location_count/201811` Path.

In [97]:
get_loc("201811")

{18: {'location': 'racecourses_and_greyhound_tracks',
  'count': 1,
  'percentage': 0.0006341154090044388},
 16: {'location': 'bus_and_coach_stations,_depots_and_companies',
  'count': 3,
  'percentage': 0.0019023462270133164},
 15: {'location': 'sports_grounds,_stadia_and_pitches',
  'count': 3,
  'percentage': 0.0019023462270133164},
 12: {'location': 'hospitals', 'count': 7, 'percentage': 0.004438807863031071},
 11: {'location': 'motorway_service_stations',
  'count': 8,
  'percentage': 0.0050729232720355105},
 10: {'location': 'further_education_establishments',
  'count': 8,
  'percentage': 0.0050729232720355105},
 17: {'location': 'shopping_centres_and_retail_parks',
  'count': 8,
  'percentage': 0.0050729232720355105},
 8: {'location': 'markets', 'count': 9, 'percentage': 0.0057070386810399495},
 13: {'location': 'conference_and_exhibition_centres',
  'count': 11,
  'percentage': 0.006975269499048827},
 9: {'location': 'theme_and_adventure_parks',
  'count': 12,
  'percentage': 

## 3. `Get_Crime`:
The `get_crime` function allows the users to extract the whole list of `crime_category` **count** for one specific date. 

`get_crime` takes in a `date` directly from the path specified in the `curl` request.

In [98]:
def get_crime(date): #"get_crime" has only one paramter ("date") which is given in the path.
    
    #Setting the API url:
    crime_url_template = 'https://data.police.uk/api/outcomes-at-location?lat={lat}&lng={lng}&date={data}'

    my_latitude = '51.509865' #The latitude of London City
    my_longitude = '-0.118092' #The longitute of London City
    extract_date = str(date) #Converting the "date" parameter given by the path to a string
    year = extract_date[0:4] #Slicing date in "year" (ex. "2018")
    month = extract_date[4:len(extract_date)] #Slicing date in "month" (ex. "11")
    my_date = year + "-" + month #Connecting "year" and "month" whilst adding a dash (ex. "2018-11")
   
    #Calling format on the API string to change {lat}, {lng}, {data} into the above-set paramters:
    crime_url = crime_url_template.format(
        lat = my_latitude,
        lng = my_longitude,
        data = my_date) 

    #Calling ".get" function to establish a connection with the Police API:
    resp = requests.get(crime_url)
    
    #If a "status_code" equal to 200 is returned then the resp in "json" format is returned:
    if resp.status_code == 200:
        dicts = resp.json()

    #Else return the "status_code" of the response:
    else:
        return resp.status_code


    #Create an empty list ("location") and an empty dictionary ("crime_category"):
    location = []
    crime_category = {}
    
    #Running a loop which iterates n_times (n = to the total records in the response dictionary):
    for i in range(len(dicts)):
        
    #For every single "i" record in the response dictionary:
    
        #1. Take the category value in the crime section of the record:
        row = dicts[i]["crime"]["category"]
        
        #2. Append the value to the location list:
        location.append(row)
        
        #3. Count the apperance of each entry:
        if row in crime_category:
            crime_category[row] = crime_category[row] +1 
        else:
            crime_category[row] = 1
  
    #Transform the dictionary with the count of the apperances of each entry into a DataFrame:
    df_crimes = pd.DataFrame(list(crime_category.items()), columns=['crime', 'count'])
    
    #Utilise the vectorised functionalities of DataFrames to compute a new percentage column:
    df_crimes["percentage"] = df_crimes["count"] / df_crimes["count"].sum()
    
    #Cleaning column strings:
    df_crimes["crime"] = clean_col(df_crimes["crime"])
    
    #Sort values in aseconding order and convert the new DataFrame back to a dictionary:
    dict_crimes = df_crimes.sort_values("percentage", ascending = True).to_dict("index")
    
    #Return the dictionary in json format:
    return dict_crimes

This is an example of an output for a request which is send to the `/crime_count/201811` Path.

In [102]:
get_crime("201811")

{13: {'crime': '', 'count': 1, 'percentage': 0.0006341154090044388},
 12: {'crime': 'other_crime', 'count': 14, 'percentage': 0.008877615726062143},
 9: {'crime': 'possession_of_weapons',
  'count': 28,
  'percentage': 0.017755231452124286},
 7: {'crime': 'vehicle_crime',
  'count': 36,
  'percentage': 0.022828154724159798},
 10: {'crime': 'criminal_damage_arson',
  'count': 40,
  'percentage': 0.025364616360177554},
 11: {'crime': 'bicycle_theft',
  'count': 40,
  'percentage': 0.025364616360177554},
 2: {'crime': 'robbery', 'count': 91, 'percentage': 0.05770450221940393},
 3: {'crime': 'theft_from_the_person',
  'count': 110,
  'percentage': 0.06975269499048826},
 4: {'crime': 'burglary', 'count': 120, 'percentage': 0.07609384908053266},
 5: {'crime': 'drugs', 'count': 124, 'percentage': 0.07863031071655041},
 6: {'crime': 'public_order', 'count': 127, 'percentage': 0.08053265694356374},
 8: {'crime': 'shoplifting', 'count': 138, 'percentage': 0.08750792644261256},
 1: {'crime': 'oth